In [1]:
import sys
import os
import pandas as pd

# Add the project root to the Python path
sys.path.append(os.path.join(os.getcwd(), '..'))

# Import the project modules
from src.config import setup_logging
from src.extract import fetch_statsbomb_event_data
from src.transform import transform_to_progressive_actions, transform_to_turnovers

# Reload modules when code is changed (uncomment for development)
%load_ext autoreload
%autoreload 2

# Init logging
logger = setup_logging(log_file="../logs/build_up_heatmaps.log")

In [2]:
events = fetch_statsbomb_event_data()
#list(events.columns)

2025-10-02 16:00:42,014 - src.extract.statsbomb_data - INFO - Fetching StatsBomb event data for Europe - UEFA Euro - 2024 - male
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/statsbombpy/api_client.py:21: No

In [3]:
prog_actions = transform_to_progressive_actions(events)
prog_actions.info()

2025-10-02 16:00:53,214 - src.transform.build_up_events - INFO - Transforming 187858 records from events data to progressive actions...
2025-10-02 16:00:53,830 - src.transform.build_up_events - INFO - Found 87760 actions (passes and carries).
2025-10-02 16:00:54,051 - src.transform.build_up_events - INFO - Found 14870 progressive actions (passes and carries).
2025-10-02 16:00:54,076 - src.transform.build_up_events - INFO - Done! Found 8771 progressive actions in own half (x < 60).


<class 'pandas.core.frame.DataFrame'>
Index: 8771 entries, 340 to 149941
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              8771 non-null   object 
 1   match_id        8771 non-null   int64  
 2   team            8771 non-null   object 
 3   player          8771 non-null   object 
 4   position        8771 non-null   object 
 5   timestamp       8771 non-null   object 
 6   x               8771 non-null   float64
 7   y               8771 non-null   float64
 8   end_x           8771 non-null   float64
 9   end_y           8771 non-null   float64
 10  progression     8771 non-null   float64
 11  type            8771 non-null   object 
 12  under_pressure  1408 non-null   object 
 13  possession      8771 non-null   int64  
dtypes: float64(5), int64(2), object(7)
memory usage: 1.0+ MB


In [4]:
turnovers = transform_to_turnovers(events)
turnovers.info()

2025-10-02 16:00:54,148 - src.transform.build_up_events - INFO - Transforming 187858 records from events data to turnovers data...
2025-10-02 16:00:55,775 - src.transform.build_up_events - INFO - Found 643 turnovers by type (Dispossessed and Miscontrol).
2025-10-02 16:00:55,794 - src.transform.build_up_events - INFO - Found 93 turnovers from 50/50s.
2025-10-02 16:00:55,873 - src.transform.build_up_events - INFO - Found 3634 turnovers from passes.
2025-10-02 16:00:55,878 - src.transform.build_up_events - INFO - Found 154 turnovers from dribbles.
2025-10-02 16:00:55,889 - src.transform.build_up_events - INFO - Found 1511 turnovers from incomplete ball receipts.
2025-10-02 16:00:55,936 - src.transform.build_up_events - INFO - Found 138 turnovers from duels when in possession.
2025-10-02 16:00:56,066 - src.transform.build_up_events - INFO - Found 6173 turnovers.
2025-10-02 16:00:56,091 - src.transform.build_up_events - INFO - Filtered out duplicates. 6173 turnovers left.


<class 'pandas.core.frame.DataFrame'>
Index: 6173 entries, 164524 to 180166
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    6173 non-null   object 
 1   match_id              6173 non-null   int64  
 2   team                  6173 non-null   object 
 3   player                6173 non-null   object 
 4   position              6173 non-null   object 
 5   timestamp             6173 non-null   object 
 6   possession            6173 non-null   int64  
 7   possession_team       6173 non-null   object 
 8   x                     6173 non-null   float64
 9   y                     6173 non-null   float64
 10  type                  6173 non-null   object 
 11  50_50                 93 non-null     object 
 12  pass_outcome          3634 non-null   object 
 13  pass_end_location     3634 non-null   object 
 14  pass_type             874 non-null    object 
 15  dribble_outcome    